## 分车型预测-使用LSTM建模预测

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from keras import backend as K
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, RNN, GRU, SimpleRNN
from sklearn.model_selection import train_test_split

np.random.seed(1904)
warnings.filterwarnings('ignore')

C:\ProgramFiles\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramFiles\Anaconda3\lib\site-packages\tensorflow\python\framew

### 导入数据

In [2]:
data_path = './data/'
train_user_reply_data = pd.read_csv(data_path + 'train_user_reply_data.csv', encoding='utf-8')
train_search_data = pd.read_csv(data_path + 'train_search_data.csv', encoding='utf-8')
train_sales_data = pd.read_csv(data_path + 'train_sales_data.csv', encoding='utf-8')
evaluation_public = pd.read_csv(data_path + 'evaluation_public.csv', encoding='utf-8')

### 提取车型和省份数据

In [3]:
cars = list(set(train_sales_data['model']))
provinces = list(set(train_sales_data['province']))
print(len(cars))
print(len(provinces))

# Normalization
train_sales_data['salesVolume'] = train_sales_data['salesVolume'].map(lambda index: np.log2(index) + 1)   # log变换
scaler = MinMaxScaler(feature_range=(0, 1))
train_sales_data[['salesVolume']] = scaler.fit_transform(train_sales_data[['salesVolume']])
print(train_sales_data[:3])

60
22
  province  adcode             model bodyType  regYear  regMonth  salesVolume
0       上海  310000  3c974920a76ac9c1      SUV     2016         1     0.557228
1       云南  530000  3c974920a76ac9c1      SUV     2016         1     0.609492
2      内蒙古  150000  3c974920a76ac9c1      SUV     2016         1     0.542952


### 构造数据

In [4]:
def generate_batch(data_set, look_back, gap_days):
    data_x, data_y = [], []
    for i in range(len(data_set) - look_back - gap_days + 1):
        a = data_set[i:(i + look_back), 0]
        data_x.append(a)
        data_y.append([data_set[i + look_back + gap_days - 1, 0]])
    return np.array(data_x), np.array(data_y), data_set[-look_back:, 0].reshape(1, look_back)

### 自定义评测指标

In [5]:
def metrics(data):
    data = scaler.inverse_transform(data)
    data = 2 ** (data - 1)
    print(data)
    
    a = data[:, 0]
    b = data[:, 1]
    res = np.sqrt(np.sum((a - b) ** 2) / len(a)) / np.mean(a)
    print('validate rmse:', 1 - res)
    return (1 - res)

### 模型构建

In [6]:
from keras.layers.advanced_activations import PReLU

def construct_lstm_model(x_train, x_test, y_train, y_test, test, car_name, batch_size):
    print('batch_size:', batch_size)
    model = Sequential()
    model.add(LSTM(32, input_shape=(1, look_back), return_sequences=True))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(1))
    model.add(PReLU())
    model.compile(loss='mean_squared_error', optimizer='adam')   # loss='mse'  'mean_squared_error'
#     model.summary()
    early_stopping = EarlyStopping('val_loss', patience=20, verbose=1)
    best_model_path = './model/' + car_name + '.h5'
    checkpoint = ModelCheckpoint(best_model_path, monitor='val_loss', mode='min', save_best_only=True, verbose=0, save_weights_only=False)
    callbacks = [checkpoint, early_stopping]
    model.fit(x_train, y_train, epochs=500, batch_size=batch_size, verbose=0, shuffle=False, callbacks=callbacks, validation_data=(x_test, y_test))
    model.load_weights(best_model_path)     # load best model parameters   shuffle=False
    predict = model.predict(test)
    predict = scaler.inverse_transform(predict)
    predict = 2 ** (predict - 1)
#     print(np.hstack((y_test, model.predict(x_test))))
    rmse = metrics(np.hstack((y_test, model.predict(x_test))))
    return predict, rmse

### 分车型&分月建模预测

In [7]:
file_name = os.listdir('./model/')
for file in file_name:
    os.remove('./model/' + file)
print(os.listdir('./model/'))

look_back = 10
label_length = 4
batch_size = 16
submit = pd.DataFrame()
flag = 0
total = 0
thresh = 1.42

for car in cars:
    labels = np.zeros([len(provinces), label_length])
    rmse_total = 0
    for k in range(1, 5, 1):
        print('{} car:{}'.format(flag+1, car))
        train = np.array([[0 for _ in range(look_back)]])
        label = np.array([[0]])
        test = np.array([[0 for _ in range(look_back)]])

        for province in provinces: 
            temp = train_sales_data[(train_sales_data['model'] == car) & (train_sales_data['province'] == province)]
    #         print(temp[['province', 'model', 'regYear', 'regMonth', 'salesVolume']])
            data = temp['salesVolume'].values.reshape(-1, 1).astype('float32')
            train_data, train_label, test_data = generate_batch(data, look_back, k)
            train = np.vstack((train, train_data))
            label = np.vstack((label, train_label))
            test = np.vstack((test, test_data))

        train = train[1:]
        label = label[1:]
        test = test[1:]
        train = np.reshape(train, (-1, 1, look_back))
        test = np.reshape(test, (-1, 1, look_back))
        x_train,x_test,y_train,y_test = train_test_split(train, label, test_size=0.1, random_state=1904)

        print('train shape:', train.shape)
        print('label shape:', label.shape)
        print('test shape:', test.shape)
        res, rmse = construct_lstm_model(x_train, x_test, y_train, y_test, test, car, int((train.shape[0] * 0.9) / 22))
        K.clear_session()    #  清除tf session
        labels[:, k-1] = res.reshape(1, -1)
        rmse_total += rmse
    print(labels)
    print('average rmse:', rmse_total / 4)
    flag += 1
    total += rmse_total / 4

    index = 0
    submit_partial = pd.DataFrame()
    for province_2 in provinces: 
        temp = evaluation_public[(evaluation_public['model'] == car) & (evaluation_public['province'] == province_2)]
        temp['forecastVolum'] = labels[index]
        submit_partial = pd.concat([submit_partial, temp], axis=0)
        index += 1
    submit = pd.concat([submit, submit_partial], axis=0)

print('all validation rmse:', total / 60)  

[]
1 car:a28bb927b6fcb33c
train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00053: early stopping
[[ 264.00000696  283.85612722]
 [  72.00000749  125.7837234 ]
 [ 246.00004064  260.90778502]
 [ 614.99985624  436.66512963]
 [  85.00001022  179.53512283]
 [ 397.00005391  400.2316373 ]
 [ 553.00010217  352.91053456]
 [ 966.00011997  494.06891349]
 [ 826.99996804  575.8824708 ]
 [ 676.99983797  548.28360617]
 [1051.00012938  882.36676129]
 [ 664.99983093  482.69388822]
 [ 330.99997547  257.53116118]
 [ 175.99997209  196.07322477]
 [3988.00086191 2817.88412695]
 [ 316.99995616  219.88468265]
 [ 220.99999662  213.16041953]
 [ 577.99998394  452.7563407 ]
 [ 316.99995616  212.18893418]
 [ 547.99987753  467.1498425 ]
 [ 466.99995201  368.97101786]
 [ 680.00008208  915.04508824]
 [ 164.99998246  142.72866468]
 [ 466.00000584  292.86609913]
 [5245.00033686 2987.73199729]
 [2391.00023935 3156.43250346]
 [ 344.99997916  428.38630127]
 [ 906.00010767  624.8

Epoch 00067: early stopping
[[1488.99989501 1125.80177224]
 [ 668.99990517  648.35257394]
 [ 352.00003806  319.13362242]
 [ 546.00012889  707.98957003]
 [ 579.00007907  637.19415999]
 [ 228.00000256  264.42550298]
 [ 139.00001451  150.6949012 ]
 [  72.00000749   81.04996911]
 [ 289.99994741  465.25287766]
 [ 181.99996076  209.04015444]
 [1385.00032736 1333.54829474]
 [ 730.00003891  621.34134123]
 [ 168.99999955  106.02433172]
 [ 388.00006849  301.56827308]
 [ 206.00002693  294.04185532]
 [ 242.00005203  328.28468495]
 [1073.99999953 1345.86189002]
 [ 513.99987826  503.13408912]
 [ 502.00003655  409.0200943 ]
 [1730.00011248 1935.85542096]
 [ 301.00004187  225.30674607]
 [ 358.000029    287.18986043]
 [ 303.00000948  267.26932118]
 [  76.99999176   60.49295038]
 [ 135.99999815  123.80952757]]
validate rmse: 0.7582905038830908
[[1539.95922852  881.82843018  654.80059814  582.18554688]
 [ 595.63537598  393.66452026  237.04420471  233.22531128]
 [2335.3894043  1509.53540039 1096.54992676 

Epoch 00088: early stopping
[[1139.99986203 1433.60179487]
 [3929.00100446 3595.1531676 ]
 [2788.99956999 3100.38140324]
 [ 466.99995201  422.6785846 ]
 [1289.99983666 1682.58454432]
 [ 462.99991421  585.49229015]
 [ 910.00016858 1222.95592261]
 [1405.99979289 1559.53320617]
 [1579.00012914 2038.66726526]
 [1034.99985319 1371.24636569]
 [1501.0003378  1645.95674442]
 [1541.00027305 1938.88660215]
 [2583.00035078 3142.70361454]
 [1881.00006956 1651.75794196]
 [6576.00150243 3823.57533764]
 [ 875.00016483 1074.98405008]
 [2280.9996254  2960.13775958]
 [ 112.0000059   170.44424641]
 [2327.99960121 1644.99624552]
 [2071.00024592 1762.44195501]
 [ 197.00001339  225.1529426 ]
 [2165.00052997 1718.74973288]
 [1562.00011824 1550.18946182]
 [3669.00046546 3918.03505321]
 [ 235.00001511  239.910856  ]
 [ 988.00009031 1061.94888644]
 [1719.99992282 1756.50200877]
 [ 667.99995115 1002.5023338 ]
 [ 674.99983638  420.95883901]]
validate rmse: 0.6404584662128263
4 car:17bc272c93f19d56
train shape: (2

train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00089: early stopping
[[188.00003701 170.22618538]
 [ 61.99999271  99.04391284]
 [323.99998087 354.93853773]
 [201.00004198 220.60991787]
 [257.00000759 312.14620423]
 [179.99999488 190.48764558]
 [271.0000524  306.51772478]
 [201.00004198 206.16789133]
 [172.99998799 127.30400154]
 [258.00000145 251.1687332 ]
 [792.99992997 659.14500542]
 [241.00001318 225.60083762]
 [103.9999999  128.51116985]
 [125.99998606 147.69766966]
 [409.00003513 478.54590508]
 [139.00001451 151.71676165]
 [155.99998713 137.5358259 ]
 [191.00001793 166.25797837]
 [145.00001233 164.57648172]
 [161.99999041 172.10636183]
 [297.00003849 253.60608211]
 [806.99991589 701.43572922]
 [ 96.99999124  86.51345952]
 [158.00002097 124.8336075 ]
 [588.99998514 460.29632622]
 [580.99988865 501.11557803]
 [175.00000947 187.25698416]
 [366.00003594 216.73951968]
 [306.00002747 257.89810872]
 [125.99998606 128.18737132]
 [108.00000242 

Epoch 00041: early stopping
[[ 306.99996437  332.8380115 ]
 [ 389.99991618  465.54811064]
 [ 267.00003576  321.6394573 ]
 [ 374.99992058  482.52794943]
 [ 512.99992205  570.01618738]
 [ 374.99992058  329.45858869]
 [1016.00026714  869.31921149]
 [  28.00000147   48.64113236]
 [ 615.99993438  469.50683719]
 [ 164.99998246  168.09723758]
 [ 357.0000797   413.5410958 ]
 [ 344.00003006  446.66296406]
 [  25.00000005   29.5739806 ]
 [ 276.00001991  299.26363401]
 [ 157.00002043  172.84309638]
 [ 145.00001233  143.47770752]
 [ 623.00000063  587.6880568 ]
 [ 608.99995221  469.30990503]
 [  54.99999863   72.90912643]
 [ 272.99999164  314.87703023]
 [  25.99999997   22.72402668]
 [ 461.99991302  460.23719488]
 [ 101.00001138  114.76691586]
 [  54.0000012    38.56042343]
 [ 117.00001197  112.60098889]]
validate rmse: 0.7864238020912387
[[392.63513184 357.80325317 359.92306519 361.628479  ]
 [239.23751831 213.682724   208.08439636 211.4699707 ]
 [450.25784302 396.32727051 398.18597412 401.8721618

...

54 car:bb9fbec9a2833839
train shape: (286, 1, 10)
label shape: (286, 1)
test shape: (22, 1, 10)
batch_size: 11
Epoch 00043: early stopping
[[ 177.00003437  320.68225692]
 [ 201.00004198  172.1638963 ]
 [ 306.99996437  300.77452439]
 [ 177.99999092  135.48077768]
 [ 299.00004546  245.74288374]
 [ 201.99996896  229.33208843]
 [ 137.00000586  166.82977146]
 [ 197.99998904  153.77779222]
 [ 271.0000524   618.89907338]
 [ 332.00000764  233.42237945]
 [1098.00001841  691.22829549]
 [ 250.99995133  258.65287589]
 [ 413.00002531  391.52973028]
 [ 139.00001451  245.5918869 ]
 [ 507.00009251  371.60937094]
 [ 145.00001233  174.29588719]
 [ 158.00002097  172.92276884]
 [ 278.99999608  315.63058847]
 [ 309.99992256  234.76599566]
 [ 863.99978947  541.41952521]
 [ 259.00002041  360.88625562]
 [ 551.99989278  713.44258158]
 [ 225.99998218  155.61086759]
 [ 398.00000325  291.26277415]
 [  91.99998961  116.90200712]
 [ 163.99999676  167.94738854]
 [ 788.00005381  541.24840116]
 [ 141.99999548  169.487

validate rmse: 0.6194276748722208
[[416.77886963 316.94555664 250.9916687  216.60211182]
 [329.5067749  229.09024048 153.4894104  127.22787476]
 [734.17077637 612.19189453 490.0453186  383.94628906]
 [367.26596069 263.60943604 172.75547791 165.2567749 ]
 [799.50189209 564.26916504 398.64764404 329.90911865]
 [174.22790527 150.01568604  84.70848846  69.36888123]
 [394.00634766 302.97488403 234.50488281 200.80966187]
 [155.93917847 146.4793396   94.23691559  77.08166504]
 [286.91473389 247.77005005 178.60704041 142.32653809]
 [117.79022217  95.92355347  67.55948639  62.83319092]
 [ 50.76799393  52.51010895  41.0992775   36.08896255]
 [307.46542358 228.83067322 160.88961792 137.93167114]
 [313.02362061 247.92388916 201.28890991 179.08999634]
 [ 76.82241058  71.59902954  45.23830795  38.85340881]
 [371.86306763 286.52981567 219.5609436  187.51663208]
 [ 72.04129028  66.17588043  42.49443817  40.53422928]
 [550.55975342 327.74502563 223.71026611 158.77915955]
 [ 64.57813263  56.71300888  35

train shape: (264, 1, 10)
label shape: (264, 1)
test shape: (22, 1, 10)
batch_size: 10
Epoch 00042: early stopping
[[1157.99984966 1036.48066764]
 [ 203.0000006   213.57717117]
 [ 895.99980879  911.13212856]
 [ 356.00007678  371.93835738]
 [ 875.99985785  728.82537118]
 [ 423.99989897  437.91693163]
 [ 229.00004815  276.78548362]
 [ 394.99989484  387.40566836]
 [ 321.0000371   400.31677494]
 [ 801.9998357   649.18813646]
 [1057.0002667  1848.879582  ]
 [ 818.99990822  615.66672263]
 [ 255.99993204  439.51004335]
 [ 469.99990502  351.4712806 ]
 [1728.99978692 1500.22679132]
 [1900.99961828 1301.64831089]
 [ 357.0000797   364.58590995]
 [ 118.00000109  229.06023902]
 [2598.00007592 1800.99647025]
 [ 388.99995138  440.00561859]
 [ 411.99994411  479.3113138 ]
 [ 515.99986545  706.89063955]
 [ 175.99997209  374.93415781]
 [ 341.0000874   455.11130731]
 [ 306.99996437  381.70477612]
 [ 328.00008099  285.90460608]
 [ 497.00001006  496.62923248]]
validate rmse: 0.5955405063107362
57 car:7a7885

59 car:63065128401bb3ff
train shape: (308, 1, 10)
label shape: (308, 1)
test shape: (22, 1, 10)
batch_size: 12
Epoch 00053: early stopping
[[1459.99968889 2199.80029304]
 [ 213.99998516  498.78147229]
 [1494.00018903 1041.22164798]
 [1218.00022926 1058.13287227]
 [ 360.99997051  678.85665299]
 [3711.00066836 2729.74707224]
 [2351.00036181 1620.76068881]
 [1223.00025475  958.11250903]
 [1219.99973282  698.76505318]
 [1466.00018231 1317.06322598]
 [3008.00059398 2897.07026666]
 [1415.00008972 1059.31864258]
 [1333.00031442 1126.74498803]
 [ 826.99996804  821.67489532]
 [1505.999987   1474.37111799]
 [1251.99998102  929.34650613]
 [ 842.00003848 1021.44387489]
 [2429.00015662 2763.65474545]
 [1747.99996423 1093.32509688]
 [1328.00003096 1254.83769995]
 [2268.99946119 2690.11645532]
 [2143.00023071 3771.99748801]
 [1191.99981106 1062.83237288]
 [1615.00034357 1139.57758229]
 [1765.99974434 1281.01453289]
 [ 624.99983587 1992.98396995]
 [2610.99966264 2408.1883986 ]
 [1980.00015659 1023.431

train shape: (242, 1, 10)
label shape: (242, 1)
test shape: (22, 1, 10)
batch_size: 9
Epoch 00044: early stopping
[[ 2418.99997376  1768.34259941]
 [ 9110.00148901  7772.02673572]
 [  748.00007014   821.0619001 ]
 [ 1113.00029505  1065.11641296]
 [ 8732.99883583  8375.61077164]
 [  769.00014934   728.68512857]
 [  429.99998057   442.55605409]
 [  957.00023291   849.25493979]
 [  486.0000612    642.13920847]
 [ 2147.00002896  1796.55394345]
 [10370.99820349  7598.54033403]
 [  950.99979103   803.30956961]
 [  321.0000371    473.85551385]
 [  821.00018808   809.30991715]
 [  469.99990502   450.3163275 ]
 [  571.99990841   482.64345738]
 [ 1828.00021201  2143.54978937]
 [ 1183.00005863   775.61076554]
 [  499.00008146   556.33003996]
 [ 2106.9998415   1827.7108181 ]
 [  423.00001424   443.86614957]
 [  225.99998218   340.76549044]
 [ 1699.00024579  1350.39806114]
 [  891.99994648   887.25612013]
 [  383.99996909   407.57508675]]
validate rmse: 0.6701498087663447
[[ 9518.97753906  9327.039

```
all validation rmse:0.6555167883490066
```

In [8]:
submit = submit.sort_values(by=['id'], ascending=True)
submit['forecastVolum'] = submit['forecastVolum'].map(lambda index: int(np.round(index)))
print('res store over')

res store over


In [9]:
sub = submit.sort_values(by=['id'], ascending=True)
print(len(sub))
sub['forecastVolum'].mean()

5280


597.4863636363636

In [10]:
sub = submit.copy()
sub['forecastVolum'] = sub['forecastVolum'] / thresh
sub['forecastVolum'] = sub['forecastVolum'].map(lambda index: int(np.round(index)))
sub.groupby(['regMonth'])['forecastVolum'].mean()
sub[['id', 'forecastVolum']].to_csv('./submit/chusai_lstm_model_1.csv', encoding='utf-8', index=None)

regMonth
1    483.454545
2    425.534091
3    393.977273
4    380.087121
Name: forecastVolum, dtype: float64